In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np # linear algebra import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd


%matplotlib inline 

import matplotlib
import matplotlib.pyplot as plt 


In [ ]:


df= pd.read_csv('/kaggle/input/covid19data/owid-covid-data.csv')
df



In [ ]:
data_in=df.loc[df['location'] == "India"]

In [ ]:

data_in["date"]= pd.to_datetime(data_in["date"]) 


In [ ]:
data_in.info()

In [ ]:
data_in.tail()

In [ ]:
data_in.drop(['total_deaths','new_deaths','gdp_per_capita','extreme_poverty','cardiovasc_death_rate','diabetes_prevalence','female_smokers','male_smokers','handwashing_facilities','hospital_beds_per_thousand','life_expectancy','human_development_index','iso_code', 'continent','total_cases_per_million','new_cases_per_million','new_cases_smoothed_per_million','total_deaths_per_million','new_tests_smoothed_per_thousand','tests_per_case','positive_rate','population','population_density','tests_units','new_deaths_smoothed_per_million', 'new_deaths_per_million','total_tests','total_tests_per_thousand','new_tests_per_thousand','new_tests_smoothed','stringency_index','median_age','aged_65_older','weekly_hosp_admissions_per_million','hosp_patients_per_million','weekly_hosp_admissions','weekly_icu_admissions_per_million','weekly_icu_admissions','reproduction_rate','hosp_patients','icu_patients_per_million','icu_patients','aged_70_older','new_tests'], axis=1, inplace=True)

In [ ]:
data_in

In [ ]:
df  = data_in[data_in['date']>'2020-03-18']
df = df[df['date']<'2020-10-28']

In [ ]:
df.drop(['location'],axis = 1,inplace = True)

In [ ]:
df

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
df.drop(['new_cases_smoothed','total_cases'],axis =1,inplace = True)

In [ ]:
df

In [ ]:
df['new_cases'].plot()

In [ ]:
plt.figure(figsize=(100,50))
plt.plot(df['date'],df['new_cases'])
plt.xlabel("date")
plt.ylabel("case")

In [ ]:
data = df['new_cases']

In [ ]:
def preprocessing(data , days=14):
    X, Y = [], []
    for i in range(len(data)-days-1):
        a = data[i:(i+days), 0]
        X.append(a)
        Y.append(data[i + days, 0])
    return np.array(X), np.array(Y)

In [ ]:

data = np.array(data)
data = data.reshape(len(data),1)
print(data.shape)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)


In [ ]:
import numpy as np 
train, test= np.split(data, [int(.8*len(data))])
train = train.reshape(len(train) , 1)
test = test.reshape(len(test) , 1)
print(test.shape)
print(train.shape)


In [ ]:
X_train,Y_train =preprocessing(train)
X_test,Y_test =preprocessing(test)

In [ ]:
Y_train = Y_train.reshape(len(Y_train),1)
Y_test = Y_test.reshape(len(Y_test),1)

In [ ]:
X_train = X_train.reshape(X_train.shape[0] , 1 ,X_train.shape[1])
X_test = X_test.reshape(X_test.shape[0] , 1 ,X_test.shape[1])

In [ ]:
import numpy
import matplotlib.pyplot as plt
import pandas
import math
from keras.models import Sequential
from keras.layers import Dense , BatchNormalization , Dropout , Activation
from keras.layers import LSTM , GRU
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras import optimizers
days =14
model = Sequential()
model.add(GRU(256 , input_shape = (1 , days) , return_sequences=True))
model.add(Dropout(0.3))
model.add(LSTM(256))
model.add(Dropout(0.3))
model.add(Dense(64 ,  activation = 'relu'))
model.add(Dense(1))
print(model.summary())

In [ ]:
optimizer = optimizers.Adam(lr=0.01)
from keras.callbacks import ReduceLROnPlateau
model.compile(loss='mean_squared_error', optimizer=optimizer , metrics = ['mean_squared_error'])
history = model.fit(X_train, Y_train, epochs=150 , batch_size = 64 , 
           validation_data = (X_test,Y_test))


In [ ]:

import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()


In [ ]:
import math
def model_score(model, X_train, y_train, X_test, y_test):
    trainScore = model.evaluate(X_train, y_train, verbose=0)
    print('Train Score: %.5f MSE (%.5f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))
    testScore = model.evaluate(X_test, y_test, verbose=0)
    print('Test Score: %.5f MSE (%.5f RMSE)' % (testScore[0], math.sqrt(testScore[0])))
    return trainScore[0], testScore[0]

model_score(model, X_train, Y_train , X_test, Y_test)


In [ ]:
pred = model.predict(X_train)
pred = scaler.inverse_transform(pred)
y_test = Y_train.reshape(Y_train.shape[0] , 1)
y_test = scaler.inverse_transform(y_test)
print("Red - Predicted,  Blue - Actual")
plt.rcParams["figure.figsize"] = (15,7)
plt.plot(y_test , 'b')
plt.plot(pred , 'r')
plt.xlabel('Time')
plt.ylabel('Stock Prices')
plt.grid(True)
plt.show()

In [ ]:
X,Y = preprocessing(data)
Y = Y.reshape(len(Y),1)
X = X.reshape(X.shape[0] , 1 ,X.shape[1])


In [ ]:
def model_graph(X,model,Y):
    pred = model.predict(X)
    pred = scaler.inverse_transform(pred)
    y_test = Y.reshape(Y.shape[0] , 1)
    y_test = scaler.inverse_transform(y_test)
    print("Red - Predicted,  Blue - Actual")
    plt.rcParams["figure.figsize"] = (15,7)
    plt.plot(y_test , 'b')
    plt.plot(pred , 'r')
    plt.xlabel('Time in days')
    plt.ylabel('Daily new deaths')
    plt.title("stacked LSTM for Daily new deaths prediction")
    plt.grid(True)
    plt.show()
    return
model_graph(X,model,Y)

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
model_b = Sequential()
model_b.add(Bidirectional(LSTM(256, activation='relu',return_sequences = True), input_shape=(21,1)))
model_b.add(Dropout(0.2))
model_b.add(Dense(256 ,  activation = 'relu'))
model_b.add(Dense(128 ,  activation = 'relu'))
model_b.add(Dense(16 ,  activation = 'relu'))
model_b.add(Dense(1 ,  activation = 'relu'))


print(model_b.summary())

In [ ]:
def split_sequence(sequence, n_steps_in, n_steps_out):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps_in
        out_end_ix = end_ix + n_steps_out
        # check if we are beyond the sequence
        if out_end_ix > len(sequence):
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)
X_train,Y_train =split_sequence(train,21,1)
X_test,Y_test =split_sequence(test,21,1)

In [ ]:
optimizer = optimizers.Adam(lr=0.01)
from keras.callbacks import ReduceLROnPlateau
model_b.compile(loss='mean_squared_error', optimizer=optimizer , metrics = ['mean_squared_error'])
history_b = model_b.fit(X_train, Y_train, epochs=4, batch_size = 64, 
           validation_data = (X_test,Y_test))

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history_b.history['loss'])
plt.plot(history_b.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:

model_score(model_b, X_train, Y_train , X_test, Y_test)


In [ ]:
p = X_test[-1,:,0].reshape(1,21,1)
pred = model_b.predict(p)
pred = pred.reshape(21,1)
total = np.append(Y,pred)
print(total.shape)

In [ ]:

print("Red - Predicted,  Blue - Actual")
plt.rcParams["figure.figsize"] = (15,7)
plt.plot(total , 'b')
plt.plot(Y , 'r')
plt.xlabel('Time in days')
plt.axvline(train.shape[0], color='g')
plt.ylabel('Daily new Cases')
plt.title("stacked LSTM for Daily new cases prediction")
plt.grid(True)
plt.show()

In [ ]:
arr = np.array(X_test[-1,:,:]).reshape(1,1,7)
print(data)
print(X_test)

In [ ]:
result = model_b.predict(arr)
new_data = arr[:,:,1:]
new_data = np.append(new_data,25)
print(new_data.shape)

In [ ]:
def predict(data,days = 100):
    final = np.array([])
    for day in range(0,days):
        result = model_b.predict(data)
        new_data = data[:,:,1:]
        new_data = np.append(new_data,result)
        data = new_data
        data = data.reshape(1,1,7)
        final = np.append(final,result[0,0,0])
    return final
final = predict(arr)
    